# Topic modeling analysis

#### Description
- All 11 qualitative questions analyzed for key topics
- Number of topics varied within the recommended range of 2 - 5 based on unique key words for each group. This was a subjective process that had to be completed by adjusting the number of components, examing the key words, looking at the intertopic distance in the visualizations and reading the most relevant responses to each topic.
- Non-cleaned text of must prominent topcs was included to make reading easier.

### Import libraries and data

In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [91]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

### Viewing the imported data

In [3]:
datadf.head(1)

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",True,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,"any, depending on the values",5.0,False,False,False,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2


In [4]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.000000,3.000000,3.000000,4.000000,3.000000,4.000000,1.000000,3.000000,5.000000,5.000000,1.000000,4.000000,5.000000,5.000000,3.000000,5.000000,2.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,4.000000,4.000000,4.0000

In [6]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 77 columns):
job_title                    726 non-null category
job_taxo                     726 non-null category
job_id                       723 non-null category
job_conduct_res              726 non-null int64
job_analyze_res              726 non-null int64
job_buy_res_report           726 non-null int64
job_manage_res_proj          726 non-null int64
job_observe_res              726 non-null int64
job_plan_res                 726 non-null int64
job_teach_res                726 non-null int64
job_advocate_res             726 non-null int64
job_hire_res_vendor          726 non-null int64
job_lead_res_team            726 non-null int64
num_employees                723 non-null category
num_researchers              722 non-null category
primary_industry             726 non-null category
types_res_used               690 non-null category
future_res                   609 non-null category
exp_con

# Analysis of qualitative questions

## 5. What is your company or organization's primary industry?

#### Clean and lemmatize the data for this question

In [5]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

#### Create word count matrix and vector. Settings: 2 word ngrams permitted; words in more than 30% of documents were ignored.

In [6]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.3)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

#### Apply LDA method using 4, 6 and 8 components (can be changed) and a random state set to ensure the results can be replicated.

In [7]:
lda5 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [8]:
lda5.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [9]:
pyLDAvis.sklearn.prepare(lda5, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.022258  0.066488       1        1  15.945548
1     -0.156636 -0.217845       2        1  15.516030
7     -0.113215  0.104614       3        1  12.905741
6      0.211751  0.084395       4        1  12.307240
4      0.146570 -0.145802       5        1  12.023908
5     -0.207465 -0.051247       6        1  10.947611
3     -0.081401  0.228426       7        1  10.516563
0      0.178137 -0.069029       8        1   9.837359, topic_info=    Category       Freq                 Term      Total  loglift  logprob
75   Default  49.000000           healthcare  49.000000  30.0000  30.0000
159  Default  41.000000           technology  41.000000  29.0000  29.0000
71   Default  31.000000           government  31.000000  28.0000  28.0000
67   Default  36.000000              fintech  36.000000  27.0000  27.0000
151  Default  38.000000             software  38.000000  26.0000  26.0000
..       ...        ...                  ...        ...      ...      ...
127   Topic8   0.125387              project   2.503123  -0.6749  -6.8859
130   Topic8   0.125677       public service   2.664812  -0.7352  -6.8836
141   Topic8   0.125673  resource technology   2.669099  -0.7368  -6.8836
145   Topic8   0.125860              science   3.415875  -0.9820  -6.8821
42    Topic8   0.125677          development   6.090174  -1.5617  -6.8836

[297 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         5  0.758497     academic
1         4  0.727381  advertising
2         1  0.456953       agency
2         2  0.130558       agency
2         3  0.195837       agency
...     ...       ...          ...
176       6  0.756087           vr
177       1  0.872702          web
178       3  0.189149         work
178       5  0.756594         work
179       5  0.758495      working

[246 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 8, 7, 5, 6, 4, 1])

#### Topic groups (4)
1. education 
2. fintech 
3. healthcare 
4. Tech 

#### Topic groups (6) 
1. fintech
2. education
3. healthcare
4. technology
5. software
6. consultancy

#### Topic groups (8) - 5 & 8 slight overlap
1. fintech
2. education
3. technology
4. consulting
5. software
6. healthcare
7. government
8. public (sector)


In [ ]:
#8 Topics

In [12]:
lda_W5 = lda5.transform(primary_industry_matrix)

In [13]:
top_doc_column5 = datadf.primary_industry.dropna()

#### Create second word count matrix. Excludes words in appear in 80%+ of all documents and in 2 or fewer documents. Include bigrams.

In [14]:
word_count_matrix5, count_vect5 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [15]:
LDA5a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA5a.fit(word_count_matrix5)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [16]:
lda_H = LDA5a.transform(word_count_matrix5)

In [17]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column5, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [10]:
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.basic_clean)
datadf.types_res_used = datadf.types_res_used.dropna().apply(nlp.lemmatize)

In [11]:
types_res_used_matrix, types_res_used_vector = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), ngram=(1,3), max_df=.3)
types_res_used_matrix, types_res_used_vector

(<690x1209 sparse matrix of type '<class 'numpy.int64'>'
 	with 7564 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [12]:
lda6 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [13]:
lda6.fit(types_res_used_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [14]:
pyLDAvis.sklearn.prepare(lda6, types_res_used_matrix, types_res_used_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.029312 -0.047520       1        1  19.583433
6     -0.113839 -0.011711       2        1  17.523106
0     -0.090529 -0.168515       3        1  16.756191
2      0.247138 -0.038751       4        1  11.931896
3      0.091311 -0.003273       5        1  11.387523
1     -0.013658  0.063420       6        1   8.793167
5     -0.031080  0.177801       7        1   7.067632
7     -0.060030  0.028549       8        1   6.957051, topic_info=     Category        Freq                Term       Total  loglift  logprob
659   Default  143.000000         qualitative  143.000000  30.0000  30.0000
693   Default   74.000000        quantitative   74.000000  29.0000  29.0000
501   Default   35.000000              market   35.000000  28.0000  28.0000
502   Default   28.000000     market research   28.000000  27.0000  27.0000
123   Default   44.000000  contextual inquiry   44.000000  26.0000  26.0000
...       ...         ...                 ...         ...      ...      ...
1146   Topic8    4.439613      user interview   37.914468   0.5206  -4.8028
560    Topic8    4.336895         observation   52.572153   0.1704  -4.8262
67     Topic8    3.858290                card   61.875585  -0.1095  -4.9432
73     Topic8    3.157960        card sorting   25.580448   0.5735  -5.1435
960    Topic8    3.403987                test   84.893212  -0.5510  -5.0684

[497 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.569035        11
0         3  0.189678        11
0         6  0.189678        11
5         1  0.684061        ab
5         3  0.201194        ab
...     ...       ...       ...
1207      2  0.654134  workshop
1207      3  0.029733  workshop
1207      6  0.029733  workshop
1207      7  0.089200  workshop
1207      8  0.029733  workshop

[832 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 7, 1, 3, 4, 2, 6, 8])

#### Topic groups (4)
1. card sort
2. contextual inquiry
3. focus group 
4. market rsearch

#### Topic groups (6) 
1. diary study
2. card (sort)
3. contextual inquiry
4. quantitative survey
5. focus (group)
6. generative (evaluative)

#### Topic groups (8) - 8 overlaps with 2 & 6
1. diary (study)
2. contextual inquiry
3. interview usability
4. gernative evaluative
5. market (research)
6. ux research
7. indepth (interview)
8. concept validation

In [ ]:
#8 Topics

In [23]:
lda_W6 = lda6.transform(types_res_used_matrix)

In [24]:
top_doc_column6 = datadf.types_res_used.dropna()

In [25]:
word_count_matrix6, count_vect6 = nlp.create_wordcount_matrix(datadf.types_res_used.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [26]:
LDA6a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA6a.fit(word_count_matrix6)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
lda_H = LDA6a.transform(word_count_matrix6)

In [28]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column6, 3)

Top 3 Documents for Topic 0: 

Document 1
qualitative interview survey card sort prototype testing usability testing analytics etc

Document 2
usability testing moderated and unmoderated card sort tree jacking max diff jtbd odi survey observational ethnographic research concept validation others a needed

Document 3
usability testing empathy1x1 interview survey card sort tree test prototype and concept testing diary study eye tracking

Top 3 Documents for Topic 1: 

Document 1
user research participant observation interview contextual enquiry usability testing feedback analysis data analysis survey

Document 2
contextual inquiry interview survey feedback form quant behavior analysis quant usability study usability testing ux workshop

Document 3
foundational product research usability testing 11 interview unmoderated user testing using usertestingcom survey contextual inquiry diary study analytics

Top 3 Documents for Topic 2: 

Document 1
qual research interview in lab moderated usabi

## 7. What types of research are you considering in the future?

In [15]:
datadf.future_res = datadf.future_res.dropna().apply(nlp.basic_clean)
datadf.future_res = datadf.future_res.dropna().apply(nlp.lemmatize)

In [16]:
future_res_matrix, future_res_vector = nlp.create_wordcount_matrix(datadf.future_res.dropna(), ngram=(1,3), max_df=.3)
future_res_matrix, future_res_vector

(<609x540 sparse matrix of type '<class 'numpy.int64'>'
 	with 3008 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [20]:
lda7 = LatentDirichletAllocation(n_components=7, random_state = 42)

In [21]:
lda7.fit(future_res_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [22]:
pyLDAvis.sklearn.prepare(lda7, future_res_matrix, future_res_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.155515 -0.019310       1        1  18.383867
0      0.098572  0.076145       2        1  17.342910
5     -0.034650  0.190805       3        1  15.360740
1      0.232345 -0.018043       4        1  14.814612
3      0.079071 -0.120589       5        1  14.021465
6     -0.094342 -0.123594       6        1  10.417755
2     -0.125480  0.014586       7        1   9.658652, topic_info=    Category        Freq         Term       Total  loglift  logprob
434  Default   70.000000        study   70.000000  30.0000  30.0000
90   Default   39.000000        diary   39.000000  29.0000  29.0000
371  Default  138.000000     research  138.000000  28.0000  28.0000
91   Default   35.000000  diary study   35.000000  27.0000  27.0000
459  Default   91.000000      testing   91.000000  26.0000  26.0000
..       ...         ...          ...         ...      ...      ...
514   Topic7    3.870691         user   38.593412   0.0377  -4.3548
10    Topic7    3.314935     analysis   22.992890   0.4006  -4.5098
204   Topic7    2.706472    interview   29.272543  -0.0437  -4.7126
228   Topic7    2.392701         like   19.558423   0.2363  -4.8358
275   Topic7    2.088149          new   17.904311   0.1886  -4.9720

[386 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         2  0.048660             ab
0         3  0.924536             ab
1         3  0.780238        ab test
2         2  0.053081     ab testing
2         3  0.902382     ab testing
...     ...       ...            ...
534       5  0.772553  web analytics
536       1  0.166979           work
536       2  0.166979           work
536       3  0.500937           work
536       5  0.083489           work

[527 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 2, 4, 7, 3])

#### Topic groups (4)
1. ab testing
2. quantitative
3. diary study
4. participatory

#### Topic groups (6) - 4 & 6 overlap
1. quantitative
2. usability testing
3. field
4. analytics
5. diary study
6. journey

#### Topic groups (8) - 5 is almost completely within 2; 6 & 8 overlap
1. quantitative
2. unmoderated usability
3. diary study
4. ab testing
5. ux
6. focus group
7. ethnographic
8. field study


In [ ]:
#8 topics

In [34]:
lda_W7 = lda7.transform(future_res_matrix)

In [35]:
top_doc_column7 = datadf.future_res.dropna()

In [36]:
word_count_matrix7, count_vect7 = nlp.create_wordcount_matrix(datadf.future_res.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [37]:
LDA7a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA7a.fit(word_count_matrix7)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [38]:
lda_H = LDA7a.transform(word_count_matrix7)

In [39]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column7, 3)


Top 3 Documents for Topic 0: 

Document 1
user experience research workshop survey interview usability testing benchmark card sorting etc and i would like to delve deeper in analytics

Document 2
doing more strategic piece to help inform the product strategy early on before the okrs are set have fundamental understanding our user in different market since we have a global product and we havent done a lot of thinking around localized content and market difference doing information architecture research

Document 3
abn multivariate testing analytic research behavioural research card sortingtree testing content auditing experience mapping information architecture ia lab testing build persona development research participant recruitment management scenariotask mapping survey design analysis usability testing userstakeholder interview workshop

Top 3 Documents for Topic 1: 

Document 1
much more usability testing heatmappingbehavior tracking along the line of what pendoio offer exploratory 

## 10. Describe your educational background with research

In [23]:
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.basic_clean)
datadf.research_educ_desc = datadf.research_educ_desc.dropna().apply(nlp.lemmatize)

In [24]:
research_educ_desc_matrix, research_educ_desc_vector = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), ngram=(1,3), max_df=.3)
research_educ_desc_matrix, research_educ_desc_vector

(<540x1118 sparse matrix of type '<class 'numpy.int64'>'
 	with 6604 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [31]:
lda10 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [32]:
lda10.fit(research_educ_desc_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [33]:
pyLDAvis.sklearn.prepare(lda10, research_educ_desc_matrix, research_educ_desc_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.021395 -0.019895       1        1  32.467903
3     -0.060932 -0.122454       2        1  25.847322
0      0.168490  0.017193       3        1  20.886478
2     -0.086164  0.125155       4        1  20.798297, topic_info=     Category        Freq         Term       Total  loglift  logprob
590   Default  129.000000       master  129.000000  30.0000  30.0000
235   Default  101.000000       degree  101.000000  29.0000  29.0000
752   Default   70.000000   psychology   70.000000  28.0000  28.0000
205   Default   90.000000       course   90.000000  27.0000  27.0000
692   Default   74.000000          phd   74.000000  26.0000  26.0000
...       ...         ...          ...         ...      ...      ...
1063   Topic4    7.622136  ux research   21.568776   0.5301  -5.2559
590    Topic4   12.352716       master  129.452406  -0.7791  -4.7731
402    Topic4    8.109497         grad   29.308146   0.2855  -5.1939
866    Topic4    8.241632       school   44.223246  -0.1098  -5.1777
235    Topic4    8.883894       degree  101.337816  -0.8639  -5.1027

[278 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.853371        10
26        1  0.574232  analysis
26        2  0.205083  analysis
26        3  0.246099  analysis
27        2  0.865786   analyst
...     ...       ...       ...
1113      1  0.375057      year
1113      2  0.250038      year
1113      3  0.218783      year
1113      4  0.156274      year
1117      2  0.823650        yr

[380 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3])

#### Topic groups (4) - no overlapping circles
1. participated
2. grad school
3. master degree
4. pyschology

#### Topic groups (6) - 1 &4, 2 & 3 overlap
1. participated
2. design research
3. social science
4. grad school
5. human factor
6. master degree

#### Topic groups (8) - extreme overlap between topics 1, 2, 4 & 8, 54% of all the responses

In [ ]:
#4 topics

In [45]:
lda_W10 = lda10.transform(research_educ_desc_matrix)

In [46]:
top_doc_column10 = datadf.research_educ_desc.dropna()

In [47]:
word_count_matrix10, count_vect10 = nlp.create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [48]:
LDA10a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA10a.fit(word_count_matrix10)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [49]:
lda_H = LDA10a.transform(word_count_matrix10)

In [50]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column10, 3)

Top 3 Documents for Topic 0: 

Document 1
i took professional development course in research up to one year in length at undergraduate level and then started postgraduate level private tuition i'd love to do a master or phd degree but am not able to manage the commitment even parttime alongside working full time i continue to take short course to increase my knowledge and develop my skill

Document 2
phd in human computer interaction research method data analysis coursework in various area eg ethnography statistic a well a supporting other people's research and eventually running my own research with supervision and advice from more senior people

Document 3
undergraduate degree in psychology extensive study of behavioral and scientific research methodology and analysis qual and quant interned and worked in lab for several year genetics neuroethology biopsychology graduate degree in architecture extensive study and practice of many type of research method

Top 3 Documents for Topic 1: 

## 14. How do you decide which events to attend?

In [34]:
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.basic_clean)
datadf.how_pick_events = datadf.how_pick_events.dropna().apply(nlp.lemmatize)

In [35]:
how_pick_events_matrix, how_pick_events_vector = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), ngram=(1,3), max_df=.3)
how_pick_events_matrix, how_pick_events_vector

(<644x1332 sparse matrix of type '<class 'numpy.int64'>'
 	with 7899 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [42]:
lda14 = LatentDirichletAllocation(n_components= 6, random_state = 42)

In [43]:
lda14.fit(how_pick_events_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [44]:
pyLDAvis.sklearn.prepare(lda14, how_pick_events_matrix, how_pick_events_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.038083  0.029819       1        1  24.486895
3     -0.103917  0.077834       2        1  17.610024
4     -0.021389 -0.024584       3        1  16.109206
1      0.130906  0.035285       4        1  14.989806
5     -0.016531 -0.132354       5        1  13.921301
0     -0.027152  0.014000       6        1  12.882768, topic_info=     Category        Freq        Term       Total  loglift  logprob
206   Default  193.000000  conference  193.000000  30.0000  30.0000
410   Default  193.000000       event  193.000000  29.0000  29.0000
1064  Default   92.000000     speaker   92.000000  28.0000  28.0000
1155  Default  166.000000       topic  166.000000  27.0000  27.0000
784   Default   23.000000  networking   23.000000  26.0000  26.0000
...       ...         ...         ...         ...      ...      ...
206    Topic6    7.502027  conference  193.775432  -1.2022  -4.9778
71     Topic6    6.377435      attend  101.035253  -0.7134  -5.1402
460    Topic6    5.113190      factor   33.023526   0.1839  -5.3612
1315   Topic6    4.972811    workshop   29.217067   0.2785  -5.3890
861    Topic6    4.621529      people   30.939371   0.1480  -5.4623

[454 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1         1  0.846176       12
10        2  0.773639   abroad
10        4  0.193410   abroad
32        1  0.072889   afford
32        2  0.510224   afford
...     ...       ...      ...
1324      2  0.174559     year
1324      3  0.116373     year
1324      4  0.252141     year
1324      5  0.019395     year
1331      2  0.772337  zealand

[769 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 5, 2, 6, 1])

#### Topic groups (4)
1. pay
2. topic
3. location
4. cost

#### Topic groups (6) - overlap between 3 & 6
1. pay
2. value
3. price
4. reputation
5. networking
6. relevance

#### Topic groups (8) - small amount of overlap among 2, 3, 4, & 6
1. design
2. affordable
3. speaker topic
4. reputation
5. location cost
6. value
7. location price
8. time away

In [ ]:
#6 topics

In [56]:
lda_W14 = lda14.transform(how_pick_events_matrix)

In [57]:
top_doc_column14 = datadf.how_pick_events.dropna()

In [58]:
word_count_matrix14, count_vect14 = nlp.create_wordcount_matrix(datadf.how_pick_events.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [59]:
LDA14a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA14a.fit(word_count_matrix14)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [60]:
lda_H = LDA14a.transform(word_count_matrix14)

In [61]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column14, 3)

Top 3 Documents for Topic 0: 

Document 1
it is usually a combination of date and my availability to travel the location the size the speaker list and whether the event seems likely to cater to more senior researcher or designer i'm at a point where traveling far for 101 level talk is just not useful to me

Document 2
depends on the cost of the event eg is it an evening meetup v a conference if it's the latter then it would depend on company training budget or depending on how relevant the topic are for my personal development goal also depending on the talk itself eg what the topic are which company are presenting i am more interested in advanced topic around strategy prioritisation or maybe new method

Document 3
location is the primary factor to consider because it will indicate whether i can afford to attend or not my company will cover cost of entry to a conference or workshop but will not cover travel or accommodation i also use personal vacation time to attend learning session s

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [45]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [46]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.3)
best_event_matrix, best_event_vector

(<594x1630 sparse matrix of type '<class 'numpy.int64'>'
 	with 8977 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [50]:
lda15 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [51]:
lda15.fit(best_event_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [52]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.034824 -0.081708       1        1  39.469059
2     -0.058105  0.069113       2        1  37.691908
1      0.092929  0.012595       3        1  22.839033, topic_info=     Category       Freq     Term      Total  loglift  logprob
1210  Default  31.000000  session  31.000000  30.0000  30.0000
303   Default  36.000000   course  36.000000  29.0000  29.0000
329   Default  58.000000      day  58.000000  28.0000  28.0000
1113  Default  23.000000     real  23.000000  27.0000  27.0000
309   Default   9.000000  covered   9.000000  26.0000  26.0000
...       ...        ...      ...        ...      ...      ...
1253   Topic3  10.938464  speaker  74.633386  -0.4436  -5.2937
1378   Topic3   9.333877     time  39.259880   0.0401  -5.4524
829    Topic3   9.090132      lot  48.785540  -0.2035  -5.4788
1116   Topic3   8.908441   really  51.404762  -0.2760  -5.4990
1471   Topic3   8.496596       ve  36.707948   0.0134  -5.5464

[243 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
10        3  0.808573        2013
45        2  0.966731  actionable
60        3  0.725296  affordable
64        1  0.601576         ago
64        2  0.370200         ago
...     ...       ...         ...
1617      1  0.451235        year
1617      2  0.472722        year
1617      3  0.064462        year
1618      1  0.636370    year ago
1618      2  0.347111    year ago

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

#### Topic groups (4) - 3 is almost entirely enclosed by 2
1. research
2. networking
3. day
4. variety

#### Topic groups (6) - 2 intersects with 1 & 4
1. think
2. practical
3. learn
4. design
5. variety
6. intimate

#### Topic groups (8) - overlap between 1 & 2 and 3 & 4
1. strive
2. world
3. concept
4. uxpa
5. immediately
6. sxsw
7. relevant
8. start


In [ ]:
#To Be Decided

In [67]:
lda_W15 = lda15.transform(best_event_matrix)

In [68]:
top_doc_column15 = datadf.best_event.dropna()

In [69]:
word_count_matrix15, count_vect15 = nlp.create_wordcount_matrix(datadf.best_event.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [70]:
LDA15a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA15a.fit(word_count_matrix15)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [71]:
lda_H = LDA15a.transform(word_count_matrix15)

In [72]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column15, 3)

Top 3 Documents for Topic 0: 

Document 1
two polar opposite experience c2mtl highly produced inspiring talk specific networking and meeting discussion and opportunity in an adjacent area to my own not my core sme so not the same old people i used what i learned from that experience to design an incredible offsite for my organization a few month after i returned a 2day immersive workshopclass on analyzing data that i took from iit many year ago it wa full of tool some of which were new to me and handson feedback not only did i walk away with tool and skill that i didn't have before i used that experience to design immersive learning course in the future it wa one of the earliest learnbydoing class that i'd taken

Document 2
the best experience i had not ux focussed wa the impact factory training workshop that wa over 2 day called 'train the trainer' fantastic and engaging trainer and really interactive session with no slide deck at all i liked the ucd london conference in 2010 or 2011 

## 16. What if any events have you attended on the subject of research in the past few years?

In [61]:
stop_words = ['nan', 'Nan', 'NaN', 'NAN']

In [62]:
stopWords = nlp.set_stop_words(stop_words)

In [77]:
stopWords

['nan',
 'Nan',
 'NaN',
 'NAN',
 'by',
 'mine',
 'when',
 'almost',
 'sometime',
 'fifteen',
 'eg',
 'besides',
 'others',
 'this',
 'so',
 'afterwards',
 'indeed',
 'toward',
 'anyone',
 'get',
 'made',
 'may',
 'here',
 'found',
 'herself',
 'often',
 'two',
 'might',
 'see',
 'how',
 'seemed',
 'own',
 'itself',
 'however',
 'beforehand',
 'therein',
 'few',
 'show',
 'whereupon',
 'while',
 'to',
 'take',
 'bottom',
 'whose',
 'was',
 'co',
 'nevertheless',
 'most',
 'where',
 'behind',
 'those',
 'their',
 'etc',
 'about',
 'across',
 'together',
 'still',
 'my',
 'except',
 'it',
 'because',
 'everyone',
 'no',
 'into',
 'myself',
 'either',
 'an',
 'move',
 'you',
 'yourself',
 'hers',
 'who',
 'well',
 'them',
 'con',
 'thus',
 'latterly',
 'describe',
 'serious',
 'within',
 'moreover',
 'before',
 'we',
 'other',
 'or',
 'among',
 'cant',
 'should',
 'forty',
 'his',
 'although',
 'bill',
 'himself',
 'every',
 'off',
 'your',
 'whither',
 'latter',
 'cry',
 'more',
 'mostly'

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

def create_wordcount_matrix(input_column, max_df=0.8, min_df=2, ngram=(1,1), stop_words='english'):
    """
    Creates a feature matrix. Matrix is as wide as the terms that meet the min/max parameters. Each document/row
    will have a wordcount for each term.
    Can find ngrams, but has default set to 1-word ngrams. Set ngrams to (1,n) to look for ngrams.
    """
    count_vect = CountVectorizer(max_df=max_df, min_df=min_df, stop_words='english', ngram_range=ngram)
    doc_term_matrix = count_vect.fit_transform(input_column.values.astype('U'))
    return doc_term_matrix, count_vect

word_count_matrix10, count_vect10 = create_wordcount_matrix(datadf.research_educ_desc.dropna(), max_df=0.8, min_df=2, ngram=(1,3), stop_words=)

In [104]:
events_attend_recent = datadf.events_attend_recent.fillna('nan')

In [106]:
events_attend_recent.shape

(726,)

In [105]:
events_attend_recent.isna().sum()

0

In [98]:
events_attend_recent = events_attend_recent.astype('str')

In [99]:
events_attend_recent = events_attend_recent.apply(nlp.basic_clean)
events_attend_recent = events_attend_recent.apply(nlp.lemmatize)

In [100]:
events_attend_recent_matrix, events_attend_recent_vector = create_wordcount_matrix(events_attend_recent, ngram=(1,3), max_df=.3, stop_words=stopWords)

In [73]:
# events_attend_recent_matrix, events_attend_recent_vector = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), ngram=(1,3), max_df=.3)
# events_attend_recent_matrix, events_attend_recent_vector

(<726x641 sparse matrix of type '<class 'numpy.int64'>'
 	with 3538 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [101]:
lda16 = LatentDirichletAllocation(n_components= 7, random_state = 42)

In [102]:
lda16.fit(events_attend_recent_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [103]:
pyLDAvis.sklearn.prepare(lda16, events_attend_recent_matrix, events_attend_recent_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.146242 -0.119383       1        1  24.413723
3     -0.030671 -0.142142       2        1  19.566921
2     -0.016173 -0.025205       3        1  17.696423
6      0.013718  0.038381       4        1  11.507298
0      0.125902  0.165655       5        1  10.872132
5     -0.062534  0.051802       6        1   8.694741
4     -0.176484  0.030892       7        1   7.248763, topic_info=    Category        Freq                Term       Total  loglift  logprob
102  Default  129.000000          conference  129.000000  30.0000  30.0000
598  Default   44.000000                uxpa   44.000000  29.0000  29.0000
142  Default   60.000000              design   60.000000  28.0000  28.0000
407  Default   14.000000                qrca   14.000000  27.0000  27.0000
320  Default   68.000000             meetups   68.000000  26.0000  26.0000
..       ...         ...                 ...         ...      ...      ...
277   Topic7    3.470976               local   64.023997  -0.2905  -4.3146
167   Topic7    2.988061                epic   40.795317   0.0104  -4.4644
2     Topic7    1.890561                2017   10.126622   0.9460  -4.9222
9     Topic7    1.672205                2019   14.435747   0.4688  -5.0449
604   Topic7    1.671258  uxpa international    4.855192   1.5579  -5.0455

[384 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
2         1  0.197499                  2017
2         2  0.296249                  2017
2         3  0.098750                  2017
2         5  0.197499                  2017
2         7  0.197499                  2017
...     ...       ...                   ...
634       2  0.863469   world usability day
635       7  0.629664             worldwide
636       7  0.834561  worldwide conference
637       1  0.776245                  year
637       4  0.194061                  year

[533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 7, 1, 6, 5])

# Topic groups (4)

1. day
2. london
3. epic
4. uxpa

#### Topic groups (6)
1. london
2. summit
3. session
4. design research
5. epic
6. uxr

#### Topic groups (8)
Heavy overlap among 6 of the 8 topics
1. local meetups
2. qrca
3. design research
4. uxpa
5. attended
6. epic
7. user research
8. focused


In [ ]:
#7 topics

In [79]:
lda_W16 = lda16.transform(events_attend_recent_matrix)

In [80]:
top_doc_column16 = datadf.events_attend_recent.dropna()

In [81]:
word_count_matrix16, count_vect16 = nlp.create_wordcount_matrix(datadf.events_attend_recent.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [82]:
LDA16a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA16a.fit(word_count_matrix16)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [83]:
lda_H = LDA16a.transform(word_count_matrix16)

In [84]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column16, 3)

Top 3 Documents for Topic 0: 

Document 1
uxr collective's strive conference in toronto

Document 2
nux manchester steve portigal's research networking dinner meetup at answerlab how to minimize research bias in your project

Document 3
strive local uxpa chapter meeting boston uxpa conference

Top 3 Documents for Topic 1: 

Document 1
researchops workshop user research sydney user research london ux insight utrecht

Document 2
research ops remote meetups ux monday in prague research tuesday in prague some ux conference meetups and barcamps in berlin bratislava brno prague and vienna

Document 3
not really research only but when i go to ux or web conference i pick research speaches nng exconf bratislava doer budapest webexpo prague ux salon tel aviv ux camp europe in berlin best thing ever mostly and i have a ticket for ux brighton

Top 3 Documents for Topic 2: 

Document 1
a 2 day ux design and research course numerous monthly nux event an nux conference in manchester

Document 2
actua

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [107]:
other_conference_types = datadf.other_conference_types.fillna('nan').apply(nlp.basic_clean)
other_conference_types = other_conference_types.apply(nlp.lemmatize)

In [119]:
other_conference_types.shape

(726,)

In [108]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(other_conference_types, ngram=(1,3), max_df=.3)
other_conference_types_matrix, other_conference_types_vector

(<726x295 sparse matrix of type '<class 'numpy.int64'>'
 	with 1136 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [112]:
lda20 = LatentDirichletAllocation(n_components= 6, random_state = 42)

In [113]:
lda20.fit(other_conference_types_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=6, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [114]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.015635  0.024319       1        1  21.176150
3      0.062237  0.053915       2        1  20.176509
5      0.073460  0.053092       3        1  18.805326
2      0.102731 -0.013472       4        1  17.408723
4     -0.037884 -0.185572       5        1  11.481833
1     -0.184909  0.067717       6        1  10.951460, topic_info=    Category       Freq        Term      Total  loglift  logprob
159  Default  14.000000        nope  14.000000  30.0000  30.0000
216  Default  35.000000     session  35.000000  29.0000  29.0000
289  Default  20.000000    workshop  20.000000  28.0000  28.0000
22   Default   6.000000        case   6.000000  27.0000  27.0000
256  Default  19.000000       think  19.000000  26.0000  26.0000
..       ...        ...         ...        ...      ...      ...
282   Topic6   1.597025          wa   7.171054   0.7098  -4.4438
33    Topic6   3.115986  conference  31.027905  -0.0866  -3.7754
200   Topic6   0.957918      really   7.347366   0.1744  -4.9550
216   Topic6   0.936724     session  35.092539  -1.4117  -4.9773
54    Topic6   0.860549      design   5.591138   0.3403  -5.0622

[293 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
0         4  0.426604                   11
0         6  0.426604                   11
1         2  0.603624             academic
1         3  0.301812             academic
2         1  0.202020             activity
...     ...       ...                  ...
289       5  0.048971             workshop
289       6  0.391771             workshop
290       1  0.422440  workshop conference
290       6  0.422440  workshop conference
293       5  0.857376                 xoxo

[447 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 6, 3, 5, 2])

#### Topic groups (4)
1. case study
2. talk
3. poster session
4. nope

#### Topic groups (6)
1. reatreat
2. working
3. quality
4. case study
5. panel discussion
6. tutorial

#### Topic groups (8) - 3 is completely inside 1; 4 and 5 share about 75% of the same area
1. working
2. case study
3. nice
4. outside
5. variety
6. multitrack
7. panel
8. method

In [ ]:
#topics

In [90]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [91]:
top_doc_column20 = datadf.other_conference_types.dropna()

In [92]:
word_count_matrix20, count_vect20 = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [93]:
LDA20a = LatentDirichletAllocation(n_components=6, random_state=42)
LDA20a.fit(word_count_matrix20)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [94]:
lda_H = LDA20a.transform(word_count_matrix20)

In [95]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column20, 3)

Top 3 Documents for Topic 0: 

Document 1
i am there to learn efficiently not socialize so the thing you didn't mention is the availability of course note at least the speakers' slide that i can study before and after the talk i have also been at conference that have a knowledgeable discussant associated with each 1hour talk the discussant ha studied the work before and asks pointed question about pro and con relationship to other technique point of failure etc when the speaker know this is coming you are le likely to get just a canned talk and you really get to the meat of the issue

Document 2
to make a conference really be incredible making something that they take away is critical cocreation is a good way to crystalize learning and having a physical artifact to take away and help you remember what you learned is great now that i wrote that i think it might be pretty cool to do a handson session on sketchnoting at the beginning of the conference so people who haven't ever done it be

## 21. Subjects you most want to see covered at a research conference

In [115]:
ideal_topics = datadf.ideal_topics.fillna('nan').apply(nlp.basic_clean)
ideal_topics = ideal_topics.apply(nlp.lemmatize)

In [116]:
ideal_topics_matrix, ideal_topics_vector = nlp.create_wordcount_matrix(ideal_topics, ngram=(1,3), max_df=.3)
ideal_topics_matrix, ideal_topics_vector

(<726x1123 sparse matrix of type '<class 'numpy.int64'>'
 	with 6020 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [117]:
lda21 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [118]:
lda21.fit(ideal_topics_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [389]:
pyLDAvis.sklearn.prepare(lda21, ideal_topics_matrix, ideal_topics_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.149435  0.004902       1        1  28.179038
1     -0.063588 -0.062169       2        1  25.641587
2     -0.030393 -0.060260       3        1  24.997328
0     -0.055453  0.117526       4        1  21.182047, topic_info=     Category        Freq                 Term       Total  loglift  logprob
989   Default   58.000000                study   58.000000  30.0000  30.0000
132   Default   51.000000                 case   51.000000  29.0000  29.0000
134   Default   43.000000           case study   43.000000  28.0000  28.0000
582   Default  126.000000               method  126.000000  27.0000  27.0000
663   Default   28.000000         organization   28.000000  26.0000  26.0000
221   Default   46.000000                 data   46.000000  25.0000  25.0000
351   Default   19.000000              finding   19.000000  24.0000  24.0000
654   Default   20.000000                  ops   20.000000  23.0000  23.0000
632   Default   14.000000           new method   14.000000  22.0000  22.0000
627   Default   61.000000                  new   61.000000  21.0000  21.0000
37    Default   14.000000                   ai   14.000000  20.0000  20.0000
826   Default   11.000000     research finding   11.000000  19.0000  19.0000
853   Default   15.000000         research ops   15.000000  18.0000  18.0000
720   Default   12.000000         presentation   12.000000  17.0000  17.0000
737   Default   18.000000              project   18.000000  16.0000  16.0000
755   Default   16.000000                quant   16.000000  15.0000  15.0000
968   Default   22.000000          stakeholder   22.000000  14.0000  14.0000
744   Default   15.000000                 qual   15.000000  13.0000  13.0000
315   Default   26.000000                ethic   26.000000  12.0000  12.0000
58    Default   14.000000          application   14.000000  11.0000  11.0000
1114  Default   12.000000              working   12.000000  10.0000  10.0000
707   Default   19.000000            practical   19.000000   9.0000   9.0000
611   Default   10.000000         mixed method   10.000000   8.0000   8.0000
610   Default   10.000000                mixed   10.000000   7.0000   7.0000
985   Default   21.000000             strategy   21.000000   6.0000   6.0000
185   Default    9.000000              content    9.000000   5.0000   5.0000
126   Default   10.000000                buyin   10.000000   4.0000   4.0000
349   Default   23.000000                field   23.000000   3.0000   3.0000
600   Default   25.000000          methodology   25.000000   2.0000   2.0000
622   Default   17.000000                 need   17.000000   1.0000   1.0000
...       ...         ...                  ...         ...      ...      ...
582    Topic4   53.245653               method  126.501881   0.6867  -3.2073
58     Topic4    8.417528          application   14.284892   1.0231  -5.0519
1029   Topic4    7.392382                think   12.538025   1.0237  -5.1818
121    Topic4   10.022700             business   20.122136   0.8550  -4.8774
727    Topic4    8.340565              problem   15.947055   0.9039  -5.0611
627    Topic4   21.828669                  new   61.422846   0.5175  -4.0990
528    Topic4    4.350216                 life    6.691423   1.1214  -5.7120
529    Topic4   11.765331                 like   28.603872   0.6636  -4.7171
441    Topic4    6.663046             industry   12.572162   0.9171  -5.2856
916    Topic4    5.995218              scaling   10.871570   0.9568  -5.3913
471    Topic4    4.281316          interesting    6.694548   1.1050  -5.7280
510    Topic4    3.517820  leadership research    5.003671   1.1997  -5.9244
262    Topic4    3.516332            difficult    5.001835   1.1996  -5.9248
634    Topic4    3.511146         new research    5.003830   1.1978  -5.9263
668    Topic4    3.509937                 orgs    5.003876   1.1974  -5.9266
509    Topic4 

#### Topic groups (4) - considerable overlap between 2 & 3
1. case study
2. finding
3. ops
4. new method

#### Topic groups (6)
1. case study
2. ops
3. stakeholder
4. new method
5. user research
6. quantitative

#### Topic groups (8)
1. case study
2. ops
3. user research
4. working
5. quant
6. mixed method
7. ethic
8. qualitative data


In [101]:
lda_W21 = lda21.transform(ideal_topics_matrix)

In [102]:
top_doc_column21 = datadf.ideal_topics.dropna()

In [103]:
word_count_matrix21, count_vect21 = nlp.create_wordcount_matrix(datadf.ideal_topics.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [104]:
LDA21a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA21a.fit(word_count_matrix21)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [105]:
lda_H = LDA21a.transform(word_count_matrix21)

In [106]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column21, 3)

Top 3 Documents for Topic 0: 

Document 1
researchops integrating research with design product and engineering growing and scaling a research practice leading a research team enterprise org structure for research team scaling the impact of research across an org research with emerging tech aimachine learning mrvrar multimodal experience etc

Document 2
facilitation skill conversation design master class teaching research basic to others skill for new research manager former ic moving into research management mixed method research design lightning improv shareout how to setup ux dashboard cxos qualinformed touchpoint specific outcome that matter most to people and ongoing quant measurement of those key experiencesmoments ethicalmoral framework for research that advance ai how do we consider downstream impact and just because we can doesnt mean we should

Document 3
i'd like to see if organization are still focusing on usability in my recent role there ha no longer been an emphasis on th

## 22. If attending a conference about research, who might you be excited to see there?

In [107]:
ideal_attendees = datadf.ideal_attendees.fillna('nan').apply(nlp.basic_clean)
ideal_attendees = ideal_attendees.apply(nlp.lemmatize)

In [108]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(ideal_attendees, ngram=(1,3), max_df=.3)
ideal_att_matrix, ideal_att_vector

(<471x923 sparse matrix of type '<class 'numpy.int64'>'
 	with 4277 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [394]:
lda22 = LatentDirichletAllocation(n_components= 4, random_state = 42)

In [395]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [396]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.099256  0.012285       1        1  26.978010
3      0.068893 -0.087438       2        1  25.507302
2     -0.020914  0.125629       3        1  25.495105
1     -0.147236 -0.050476       4        1  22.019582, topic_info=    Category       Freq              Term      Total  loglift  logprob
353  Default  40.000000          industry  40.000000  30.0000  30.0000
920  Default  16.000000             young  16.000000  29.0000  29.0000
514  Default  18.000000               new  18.000000  28.0000  28.0000
226  Default  14.000000             erika  14.000000  27.0000  27.0000
349  Default  12.000000        indi young  12.000000  26.0000  26.0000
348  Default  12.000000              indi  12.000000  25.0000  25.0000
227  Default  13.000000        erika hall  13.000000  24.0000  24.0000
264  Default  31.000000             field  31.000000  23.0000  23.0000
784  Default  22.000000             steve  22.000000  22.0000  22.0000
244  Default  22.000000        experience  22.000000  21.0000  21.0000
308  Default  15.000000              hall  15.000000  20.0000  20.0000
532  Default   8.000000            norman   8.000000  19.0000  19.0000
830  Default   8.000000             tomer   8.000000  18.0000  18.0000
406  Default  15.000000            ladner  15.000000  17.0000  17.0000
705  Default  15.000000               sam  15.000000  16.0000  16.0000
706  Default  15.000000        sam ladner  15.000000  15.0000  15.0000
687  Default   9.000000     research team   9.000000  14.0000  14.0000
601  Default  18.000000          portigal  18.000000  13.0000  13.0000
464  Default  11.000000              love  11.000000  12.0000  12.0000
811  Default  20.000000              tech  20.000000  11.0000  11.0000
734  Default   7.000000            sharon   7.000000  10.0000  10.0000
786  Default  17.000000    steve portigal  17.000000   9.0000   9.0000
422  Default   8.000000      leader field   8.000000   8.0000   8.0000
419  Default  54.000000            leader  54.000000   7.0000   7.0000
445  Default  43.000000              like  43.000000   6.0000   6.0000
153  Default  11.000000              data  11.000000   5.0000   5.0000
831  Default   6.000000      tomer sharon   6.000000   4.0000   4.0000
520  Default   6.000000           nielsen   6.000000   3.0000   3.0000
544  Default  32.000000      organization  32.000000   2.0000   2.0000
650  Default   9.000000              real   9.000000   1.0000   1.0000
..       ...        ...               ...        ...      ...      ...
293   Topic4   2.637633  google microsoft   3.270208   1.2983  -5.9249
812   Topic4   5.911240      tech company   7.362999   1.2936  -5.1180
425   Topic4   5.088393           leading   6.543947   1.2617  -5.2678
509   Topic4   1.830903         nate bolt   2.456270   1.2194  -6.2900
718   Topic4   1.830903             scott   2.456270   1.2194  -6.2900
865   Topic4   5.861215     user research   8.188759   1.1788  -5.1265
784   Topic4  14.034420             steve  22.179972   1.0556  -4.2533
601   Topic4  10.760934          portigal  18.085203   0.9941  -4.5189
35    Topic4   4.163990    anthropologist   5.738417   1.1925  -5.4683
786   Topic4  10.012089    steve portigal  17.270355   0.9680  -4.5910
406   Topic4   8.592003            ladner  15.663440   0.9127  -4.7440
705   Topic4   8.592003               sam  15.663440   0.9127  -4.7440
706   Topic4   8.592003        sam ladner  15.663440   0.9127  -4.7440
773   Topic4   7.349209             spool  13.995644   0.8691  -4.9002
339   Topic4   4.294067              ideo   6.565608   1.0886  -5.4376
202   Topic4   8.297746               don  17.304115   0.7783  -4.7788
811   Topic4   8.716443              tech  20.675885   0.6495  -4.7296
112   Topic4  13.352545           company  49.728987   0.1984  -4.3031
547   Topic4   6.047930              orgs  11.536891   0.8674  -5.0951
783   Topic4

#### Topic groups (4)
1. industry
2. field
3. erika hall
4. Indi young

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. steve portigal
2. expert
3. diverse
4. new
5. startup
6. doing research

#### Topic groups (8)
1. google
2. jared spool
3. different
4. new
5. consultant
6. erika hall
7. practioner
8. steve portigal


In [112]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [113]:
top_doc_column22 = datadf.ideal_attendees.dropna()

In [114]:
word_count_matrix22, count_vect22 = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [115]:
LDA22a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA22a.fit(word_count_matrix22)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [116]:
lda_H = LDA22a.transform(word_count_matrix22)

In [117]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column22, 3)

Top 3 Documents for Topic 0: 

Document 1
people with experience in building company by applying research knowledge i think there are people out there speaking from a consulting view they donat take a many risk a the one iad like to learn from company are usually led by people with a techdesign background if research is creating a real value in company and if we believe research can lead the strategic vision of a company iall be excited to see people who have managed to achieve that

Document 2
i would like to hear from and network with other organizational leader i want to meet people i can learn from no pr or salestype presentation no product pushing although i'd like to learn about tool that people are using i am much more interested in strategy and how to manage operation for large team also speaking a someone on the consulting side it would be nice to hear from leader in that space a well a leader within organization some of the advice from design executive doesn't really apply to

# Recommendations

In [383]:
recommendations = datadf.recommendations.fillna('nan').apply(nlp.basic_clean)
recommendations = recommendations.apply(nlp.lemmatize)

In [382]:
recommendations_matrix, recommendations_vector = nlp.create_wordcount_matrix(recommendations, ngram=(1,3), max_df=.3)
recommendations_matrix, recommendations_vector

(<494x1409 sparse matrix of type '<class 'numpy.int64'>'
 	with 7225 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.3, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [407]:
lda23 = LatentDirichletAllocation(n_components= 8, random_state = 42)

In [408]:
lda23.fit(recommendations_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [409]:
pyLDAvis.sklearn.prepare(lda23, recommendations_matrix, recommendations_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.075875 -0.005570       1        1  18.590069
5      0.061837  0.052963       2        1  17.491367
6      0.058806  0.004421       3        1  15.921481
3      0.003337 -0.031329       4        1  14.333301
2     -0.007261 -0.016992       5        1  13.461363
1      0.020556  0.025219       6        1   8.967949
4     -0.077382 -0.129276       7        1   6.142941
0     -0.135768  0.100564       8        1   5.091529, topic_info=     Category        Freq             Term       Total  loglift  logprob
735   Default  118.000000             make  118.000000  30.0000  30.0000
528   Default   22.000000             help   22.000000  29.0000  29.0000
895   Default  130.000000           people  130.000000  28.0000  28.0000
217   Default  115.000000       conference  115.000000  27.0000  27.0000
77    Default   35.000000           attend   35.000000  26.0000  26.0000
497   Default   29.000000            great   29.000000  25.0000  25.0000
348   Default   73.000000              don   73.000000  24.0000  24.0000
736   Default    9.000000  make accessible    9.000000  23.0000  23.0000
1327  Default   28.000000               ux   28.000000  22.0000  22.0000
12    Default   17.000000       accessible   17.000000  21.0000  21.0000
35    Default   10.000000       affordable   10.000000  20.0000  20.0000
769   Default   14.000000            maybe   14.000000  19.0000  19.0000
947   Default   17.000000        practical   17.000000  18.0000  18.0000
400   Default   23.000000           events   23.000000  17.0000  17.0000
489   Default   40.000000             good   40.000000  16.0000  16.0000
1397  Default   17.000000            world   17.000000  15.0000  15.0000
193   Default   11.000000             come   11.000000  14.0000  14.0000
396   Default   10.000000           europe   10.000000  13.0000  13.0000
1116  Default   22.000000         sessions   22.000000  12.0000  12.0000
1193  Default   10.000000            stuff   10.000000  11.0000  11.0000
834   Default    5.000000            north    5.000000  10.0000  10.0000
1329  Default   10.000000      ux research   10.000000   9.0000   9.0000
844   Default    6.000000         offering    6.000000   8.0000   8.0000
1224  Default   27.000000             talk   27.000000   7.0000   7.0000
230   Default   65.000000      conferences   65.000000   6.0000   6.0000
1146  Default   12.000000          smaller   12.000000   5.0000   5.0000
1362  Default   28.000000             want   28.000000   4.0000   4.0000
1067  Default   49.000000      researchers   49.000000   3.0000   3.0000
458   Default   15.000000             food   15.000000   2.0000   2.0000
710   Default   12.000000             look   12.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
1155   Topic8    0.808128           spaces    2.353252   1.9088  -6.2079
15     Topic8    0.808122              act    2.353253   1.9088  -6.2079
181    Topic8    0.808121  city conference    2.353253   1.9088  -6.2079
193    Topic8    3.681866             come   11.203241   1.8648  -4.6915
528    Topic8    6.923221             help   22.873666   1.7825  -4.0600
1102   Topic8    1.526172             sell    4.811921   1.8293  -5.5721
1185   Topic8    1.526606           strive    4.856192   1.8204  -5.5718
1248   Topic8    1.526676           theory    4.866986   1.8182  -5.5718
769    Topic8    3.927078            maybe   14.851469   1.6474  -4.6270
497    Topic8    6.902519            great   29.733630   1.5172  -4.0630
64     Topic8    2.245028             area    8.116479   1.6924  -5.1862
396    Topic8    2.629422           europe   10.206414   1.6213  -5.0281
710    Topic8    2.963444             look   12.981362   1.5004  -4.9085
895    Topic8   12.152369           people  130.841813   0.6011  -3.4973
217    Topic8    7.394101       conference  115.865740   0.

#### Topic groups (4)
1. pratical
2. affordable
3. expensive
4. levels

#### Topic groups (6) - some overlap between 2, 3, & 4; 5 is mostly overlapped with 2 & 4
1. ux
2. interesting
3. content
4. food
5. learning
6. affordalbe

#### Topic groups (8) - significant overlap amoung 6 of the 8 groups
1. events
2. ux
3. world
4. new
5. food
6. diverse
7. make accessible
8. north america

In [410]:
lda_W23 = lda23.transform(recommendations_matrix)

In [411]:
top_doc_column23 = datadf.recommendations.dropna()

In [412]:
word_count_matrix23, count_vect23 = nlp.create_wordcount_matrix(datadf.recommendations.dropna(), max_df=0.8, min_df=2, ngram=(1,3))

In [413]:
LDA23a = LatentDirichletAllocation(n_components=5, random_state=42)
LDA23a.fit(word_count_matrix23)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [414]:
lda23.fit(recommendations_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=8, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [415]:
lda_H = LDA23a.transform(word_count_matrix23)

In [416]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column23, 3)

Top 3 Documents for Topic 0: 

Document 1
the deeper you go into the subject matter the better a lot of research literature is either design adjacent or academic focusing on a research practitioner daytoday someone that is in a software company or an agency or a startup would be super helpful anything that help me grow my skill and competency to ultimately be a better researcher

Document 2
put together a draft conference agenda go out to the community and research what option people desire the most use the insight to inform a single track conference which you can take around the world to multiple location so it accessible on a global scale keep it small and intimate make it feel like a community event repeat each year

Document 3
ha ha ha i wonder how this would be similar to or different from epic i like the idea of a small intimate conference that focus on interactive session and workshop rather than paper there is a place for that too but i feel there are enough of those i'd love a